# Torrent to Google Drive

Simple cell-by-cell workflow. Run cells in order.

In [ ]:
import sys
import subprocess
subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'libtorrent==2.0.11'])
print('✅ Dependencies installed')

In [ ]:
import libtorrent as lt
import time
import os
import shutil

SAVE_PATH = '/content/torrents'
os.makedirs(SAVE_PATH, exist_ok=True)
print(f'✅ Save path: {SAVE_PATH}')

In [ ]:
MAGNET_LINK = ''

if not MAGNET_LINK:
    print('⚠️ Set MAGNET_LINK variable above')
else:
    print('✅ Magnet link set')

In [ ]:
trackers = [
    'udp://tracker.opentrackr.org:1337/announce',
    'udp://tracker.torrent.eu.org:451/announce',
    'udp://exodus.desync.com:6969/announce',
    'udp://tracker.openbittorrent.com:6969/announce'
]
magnet = MAGNET_LINK
for tr in trackers:
    magnet += f'&tr={tr}'

session = lt.session({'enable_dht': True, 'enable_lsd': True})
params = lt.parse_magnet_uri(magnet)
params.save_path = SAVE_PATH
handle = session.add_torrent(params)

print('📡 Fetching metadata...')
while not handle.status().has_metadata:
    time.sleep(1)

print(f'✅ {handle.status().name}')
print(f'📦 Size: {handle.status().total_wanted/(1024**3):.2f} GB')

In [ ]:
print('⬇️ Downloading...')
while not handle.status().is_seeding:
    s = handle.status()
    print(f'\r{s.progress*100:.1f}% | ↓{s.download_rate/1024:.0f} KB/s | Peers: {s.num_peers}', end='')
    time.sleep(2)

print('\n🎉 Download complete!')

In [ ]:
from google.colab import auth
import google.auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

auth.authenticate_user()
creds, _ = google.auth.default()
service = build('drive', 'v3', credentials=creds, cache_discovery=False)
print('✅ Authenticated with Google Drive')

In [ ]:
files = [f for f in os.listdir(SAVE_PATH) if not f.startswith('.')]
if not files:
    print('❌ No files found')
else:
    for i, f in enumerate(files):
        size = os.path.getsize(os.path.join(SAVE_PATH, f))/(1024**2)
        print(f'{i}: {f} ({size:.1f} MB)')

In [ ]:
FILE_INDEX = 0
FOLDER_NAME = 'Torrent'

file_path = os.path.join(SAVE_PATH, files[FILE_INDEX])

query = f"mimeType='application/vnd.google-apps.folder' and name='{FOLDER_NAME}' and trashed=false"
results = service.files().list(q=query, fields='files(id)', pageSize=1).execute()
folders = results.get('files', [])

if folders:
    folder_id = folders[0]['id']
else:
    folder_meta = {'name': FOLDER_NAME, 'mimeType': 'application/vnd.google-apps.folder'}
    folder = service.files().create(body=folder_meta, fields='id').execute()
    folder_id = folder['id']

print(f'⬆️ Uploading {files[FILE_INDEX]}...')
file_metadata = {'name': files[FILE_INDEX], 'parents': [folder_id]}
media = MediaFileUpload(file_path, chunksize=10*1024*1024, resumable=True)
request = service.files().create(body=file_metadata, media_body=media, fields='webViewLink')

response = None
while response is None:
    status, response = request.next_chunk()
    if status:
        print(f'\r{int(status.progress()*100)}%', end='')

print(f'\n✅ Upload complete!')
print(f'🔗 {response.get("webViewLink")}')